In [ ]:
import sys
import pandas as pd
from lxml import etree
import re

print(sys.executable)

## Preprocessing

1. extration des articles à lier et des features utiles dans df
2. chargement du référentiel des communes (COG 2011) dans le df main_insee_commune

In [ ]:
# Extraction des features de chaque article à lier dans un df

xml_file = '../../data/PO_t7/PO_t7.xml'
xml_tree = etree.parse(xml_file)

data = []

i = 0 # compter les articles

for article in xml_tree.xpath('//article'):
    i+=1
    # on ne retient pas les lieux hors France
    if article.find('./localisationpa') is not None:
        continue
    vedettes = article.xpath('./vedette/i')
    # on ne retient que la première vedette
    vedette = vedettes[0].text if vedettes else ''

    # on ne retient que le premier dpt de localisation
    localisationde = article.find('./localisationde')
    localisationde = localisationde.text if localisationde is not None else 'none'

    # on ne retient que le premier canton de localisation
    localisationca = article.find('./localisationca')
    if localisationca is not None:
        localisationca = re.search(
            r'<localisationca>(.*?)<\/localisationca>',
            etree.tostring(localisationca, encoding=str)
        ).group(1)
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        '''
        localisationca = re.sub(r"c<sup>on</sup> (de |d’|de la |du |de l’|des )", '', localisationca)
    else:
        localisationca = 'none'

    #  on ne retient que la première commune de localisation
    localisationco = article.find('./localisationco')
    if localisationco is not None:
        localisationco = re.search(
            r'<localisationco>(.*?)<\/localisationco>',
            etree.tostring(localisationco, encoding=str)
        ).group(1)
        '''
        cases:
        - 'c<sup>on</sup> de '
        - 'c<sup>on</sup> du '
        - 'c<sup>on</sup> d’'
        - 'c<sup>on</sup> de la '
        - 'c<sup>on</sup> de l’'
        - 'c<sup>on</sup> des '
        - 'c<sup>nes</sup> de '
        - 'c<sup>nes</sup> d’'
            '''
        localisationco = re.sub(r"c<sup>ne(s)?</sup> (de |d’|de la |du |de l’|des )", '', localisationco)

    else:
        localisationco = 'none'

    # Extraire le contenu textuel de la balise "article"
    article_text = re.sub(r'\W', ' ', article.xpath('string(.)'))

    # on insère tout dans un dictionnaire
    data.append({
        'old-id': article.get('old-id'),
        'vedette': vedette,
        'localisationde': localisationde,
        'localisationca': localisationca,
        'localisationco': localisationco,
        'reference': article_text  # Ajouter la colonne "reference"
    })

df = pd.DataFrame(data)

'''
article : nombre d’articles dans la source => `//article`
article_to_link : nombre d’articles à lier (en France) => `//article[not(localisationpa)]`
article_commune : nombre d’articles de type commune => `//article[not(localisationpa) and not(localisationco)]`
article_loc_com : nombre d’articles localisés dans une commune => `//article[not(localisationpa) and localisationco]`
'''
report = dict(
    article = i,
    article_to_link = len(df.index),
    article_commune = (df.localisationco == 'none').sum(),
    article_loc_com = (df.localisationco != 'none').sum()
)

# check
[print(k,':',v) for k, v in report.items()]
df.iloc[1500:1510, :]

In [ ]:
# Ajout du dpt code au df
dpt_df = pd.read_csv('departements-region.csv')
dpt_dict = dict(zip(dpt_df['dep_name'], dpt_df['num_dep']))

# on ajoute notre colonne avec les numéros des départements qui matchent
df['dpt_code'] = df['localisationde'].apply(lambda x: dpt_dict.get(x, 'none'))

#on renomme et réordonne pour plus de clarté.
df = df.rename(columns={'old-id' : 'article_id', 'localisationca' : 'canton_code', 'localisationco_present' : 'nom_commune'})
df = df[['article_id', 'vedette', 'localisationde', 'dpt_code', 'canton_code', 'localisationco', 'reference']]

df.iloc[1500:1510, :]

In [ ]:
# Chargement du référentiel des communes
fields = ['insee_code', 'DEP_id', 'NCCENR']
main_insee_commune = pd.read_csv("main_insee_commune.tsv",
                                 delimiter='\t',
                                 usecols=fields,
                                 dtype={'insee_code': 'string'})
main_insee_commune['DEP_id'] = main_insee_commune['DEP_id'].apply(lambda x: x[4:])
main_insee_commune.head(5)

In [ ]:
# Liste les départements présents pour mieux filtrer quand pas de localisationde
dpt_list = df['dpt_code'].unique().tolist()
dpt_list.remove("none")
dpt_list.sort()
print(dpt_list)

## Liage des articles de type commune

In [ ]:
# dataframe à charger
linked_places_df = pd.DataFrame()

In [ ]:
# Extraction des articles de type commune (ceux qui n’ont pas de commune de localisation)

communes_df = df[df['localisationco'] == 'none']
communes_df.iloc[1500:1510, :]
#communes_df.shape

### Exact match method

#### With dpt

In [ ]:
# liage des communes avec dpt de localisation (dpt/exact match)
communes_exact_dpt_df = pd.merge(communes_df,
                                 main_insee_commune,
                                 how='inner',
                                 left_on=['vedette','dpt_code'],
                                 right_on = ['NCCENR','DEP_id'])
communes_exact_dpt_df = communes_exact_dpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

communes_exact_dpt_df['method'] = 'dpt_exact'

# actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_exact_dpt_df]).drop_duplicates()

In [ ]:
# Tests
#communes_exact_dpt_df.head(5)
communes_exact_dpt_df.shape

In [ ]:
# des doublons de liage exact match dans un même dpt ?
#communes_exact_dpt_df.article_id.duplicated().sum()
communes_exact_dpt_df.loc[communes_exact_dpt_df.duplicated(keep=False, subset=['article_id']), :]

#### Without dpt

In [ ]:
# liage des communes sans dpt de localisation (nodpt/exact match)
communes_exact_nodpt_df = pd.merge(communes_df.loc[communes_df['dpt_code'] == 'none'],
                                   main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                   how='inner',
                                   left_on='vedette',
                                   right_on = 'NCCENR')
communes_exact_nodpt_df = communes_exact_nodpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

communes_exact_nodpt_df['method'] = 'nodpt_exact'

# actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_exact_nodpt_df]).drop_duplicates()

In [ ]:
# Test
communes_exact_nodpt_df
#linked_places_df.shape
communes_exact_nodpt_df.agg(['nunique', 'count', 'size'])

In [ ]:
communes_exact_nodpt_df.loc[communes_exact_nodpt_df.duplicated(keep=False,subset=['article_id']), :]

### Fuzzy method

#### With dpt

In [ ]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes avec dpt de localisation (dpt/fuzzy match)
communes_fuzzy_dpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code != 'none']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['vedette']
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df.apply(
    lambda x: [f"{nccenr} ({nccenr_to_insee.get(nccenr, '')})" for nccenr in difflib.get_close_matches(x['vedette'], main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR'], n=3)],
    axis=1
)

# Sortir les communes non liées
communes_fuzzy_dpt_df = communes_fuzzy_dpt_df[communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
communes_fuzzy_dpt_df['insee_code'] = communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_dpt_df]).drop_duplicates()



In [ ]:
# Tests
communes_fuzzy_dpt_df.head(20)
#linked_places_df.shape
#linked_places_df
communes_fuzzy_dpt_df.agg(['nunique', 'count', 'size'])

#### Without dpt

In [ ]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes sans dpt de localisation (fuzzy match)
communes_fuzzy_nodpt_df = communes_df[~communes_df.article_id.isin(linked_places_df['article_id'])][communes_df.dpt_code == 'none']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['vedette']
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df.apply(
    lambda x: [f"{nccenr} ({nccenr_to_insee.get(nccenr, '')})" for nccenr in difflib.get_close_matches(x['vedette'], main_insee_commune['NCCENR'], n=3)],
    axis=1
)

# Sortir les communes non liées
communes_fuzzy_nodpt_df = communes_fuzzy_nodpt_df[communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
communes_fuzzy_nodpt_df['insee_code'] = communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, communes_fuzzy_nodpt_df]).drop_duplicates()



In [ ]:
# Tests
communes_fuzzy_nodpt_df
#communes_fuzzy_nodpt_df.shape
#linked_places_df.shape
communes_fuzzy_nodpt_df.agg(['nunique', 'count', 'size'])

## Liage des communes de localisation

In [ ]:
# Extraction des articles appartenant à une commune (ceux ont une commune de localisation)

pas_communes_df = df[df['localisationco'] != 'none']
pas_communes_df

### Exact match method

#### With dpt

In [ ]:
# liage des communes avec localisation (dpt/exact match)
pas_communes_exact_dpt_df = pd.merge(pas_communes_df,
                                 main_insee_commune,
                                 how='inner',
                                 left_on=['localisationco','dpt_code'],
                                 right_on = ['NCCENR','DEP_id'])
pas_communes_exact_dpt_df = pas_communes_exact_dpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

pas_communes_exact_dpt_df['method'] = 'dpt_exact'

# actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_exact_dpt_df]).drop_duplicates()

In [ ]:
# Tests
pas_communes_exact_dpt_df.head(5)
#pas_communes_exact_dpt_df.shape
pas_communes_exact_dpt_df.agg(['nunique', 'count', 'size'])

#### Without dpt

In [ ]:
# liage des communes sans dpt de localisation (nodpt/exact match)
pas_communes_exact_nodpt_df = pd.merge(pas_communes_df.loc[pas_communes_df['dpt_code'] == 'none'],
                                   main_insee_commune[main_insee_commune.DEP_id.isin(dpt_list)],
                                   how='inner',
                                   left_on='localisationco',
                                   right_on = 'NCCENR')
pas_communes_exact_nodpt_df = pas_communes_exact_nodpt_df.drop(columns=['DEP_id', 'NCCENR'], axis=1)

pas_communes_exact_nodpt_df['method'] = 'nodpt_exact'

# actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_exact_nodpt_df]).drop_duplicates()

In [ ]:
# Test
pas_communes_exact_nodpt_df
#linked_places_df.shape
pas_communes_exact_nodpt_df.agg(['nunique', 'count', 'size'])

### Fuzzy match method

#### With dpt

In [ ]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes avec dpt de localisation (dpt/fuzzy match)
pas_communes_fuzzy_dpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code != 'none']
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['localisationco']
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df.apply(
    lambda x: [f"{nccenr} ({nccenr_to_insee.get(nccenr, '')})" for nccenr in difflib.get_close_matches(x['localisationco'], main_insee_commune[main_insee_commune['DEP_id'] == x['dpt_code']]['NCCENR'], n=3)],
    axis=1
)

# Sortir les communes non liées
pas_communes_fuzzy_dpt_df = pas_communes_fuzzy_dpt_df[pas_communes_fuzzy_dpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
pas_communes_fuzzy_dpt_df['insee_code'] = pas_communes_fuzzy_dpt_df['insee_code'].apply(lambda x: ', '.join(x))

pas_communes_fuzzy_dpt_df['method'] = 'dpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_dpt_df]).drop_duplicates()



In [ ]:
# Tests
pas_communes_fuzzy_dpt_df
#linked_places_df.shape
#linked_places_df
pas_communes_fuzzy_dpt_df.agg(['nunique', 'count', 'size'])

#### Without dpt

In [ ]:
import difflib

# Créer un dictionnaire de correspondance entre les NCCENR et les codes INSEE
nccenr_to_insee = main_insee_commune.set_index('NCCENR')['insee_code'].to_dict()

# Lier les communes sans dpt de localisation (fuzzy match)
pas_communes_fuzzy_nodpt_df = pas_communes_df[~pas_communes_df.article_id.isin(linked_places_df['article_id'])][pas_communes_df.dpt_code == 'none']
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['localisationco']
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df.apply(
    lambda x: [f"{nccenr} ({nccenr_to_insee.get(nccenr, '')})" for nccenr in difflib.get_close_matches(x['localisationco'], main_insee_commune['NCCENR'], n=3)],
    axis=1
)

# Sortir les communes non liées
pas_communes_fuzzy_nodpt_df = pas_communes_fuzzy_nodpt_df[pas_communes_fuzzy_nodpt_df['insee_code'].map(lambda x: len(x)) > 0]

# Concaténer les codes INSEE avec les NCCENR dans la même cellule
pas_communes_fuzzy_nodpt_df['insee_code'] = pas_communes_fuzzy_nodpt_df['insee_code'].apply(lambda x: ', '.join(x))

pas_communes_fuzzy_nodpt_df['method'] = 'nodpt_fuzzy'

# Actualiser le df des liages réalisés
linked_places_df = pd.concat([linked_places_df, pas_communes_fuzzy_nodpt_df]).drop_duplicates()

In [ ]:
# Tests
pas_communes_fuzzy_nodpt_df
#linked_places_df.shape
pas_communes_fuzzy_nodpt_df.agg(['nunique', 'count', 'size'])

## Rapport et analyse

In [ ]:
#on append les deux dataframes ensemble

new_rows_df = df[~df['article_id'].isin(linked_places_df['article_id'])]

if 'method' not in new_rows_df.columns:
    new_rows_df['method'] = 'nulle'

linked_places_df = linked_places_df.append(new_rows_df, ignore_index=True)


In [ ]:
linked_places_df.agg(['nunique', 'count', 'size'])

In [ ]:
# des doublons?
nombre_de_doublons = linked_places_df.duplicated(subset=['article_id']).sum()
print(nombre_de_doublons)


In [ ]:
linked_places_df.loc[linked_places_df.duplicated(keep=False,subset=['article_id']), :]

In [ ]:
method_counts = linked_places_df['method'].value_counts()
print(method_counts)

In [ ]:
vedette_list = pd.unique(linked_places_df['vedette'])
linked_places_df['sort_key'] = linked_places_df.apply(lambda x: x['localisationco'] if x['localisationco'] in vedette_list else x['vedette'], axis=1)
linked_places_df.sort_values(['method', 'sort_key', 'article_id'], inplace=True)
linked_places_df = linked_places_df.drop('sort_key', axis=1)
linked_places_df = linked_places_df[['article_id', 'vedette', 'localisationde', 'dpt_code', 'canton_code', 'localisationco', 'insee_code', 'method', 'reference']]
linked_places_df

## Export

In [ ]:
# Sauvegarder le dataframe fusionné
#linked_places_df.to_csv('../../utils/pouille_linking/out/liage_po7.csv', index=False)
